In [1]:
import keras
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,BatchNormalization
from keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
%matplotlib inline

Using TensorFlow backend.


In [2]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value. later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (f, f), strides = (1,1), padding = "same", 
               name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = "same", 
               name = conv_name_base + "2b", kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(F1, (f, f), strides = (s,s), padding = "same",
               name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X = Conv2D(F2, (f, f), strides = (1,1), padding = "same",
               name = conv_name_base + '2b',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)

    X_shortcut = Conv2D(F2, (f, f), strides = (s,s), padding = "same",
                        name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [4]:
def ResNet56(input_shape = (32, 32, 3), classes = 10):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Zero-Padding
    
    # Stage 1
    X = Conv2D(16, (3, 3), strides = (1, 1), padding = "same", name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis = 1, name = 'bn_conv1')(X)
    X = Activation('relu')(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [16, 16], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [16, 16], stage=2, block='b')
    X = identity_block(X, 3, [16, 16], stage=2, block='c')
    X = identity_block(X, 3, [16, 16], stage=2, block='d')
    X = identity_block(X, 3, [16, 16], stage=2, block='e')
    X = identity_block(X, 3, [16, 16], stage=2, block='f')
    X = identity_block(X, 3, [16, 16], stage=2, block='g')
    X = identity_block(X, 3, [16, 16], stage=2, block='h')
    X = identity_block(X, 3, [16, 16], stage=2, block='i')

    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [32, 32], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [32, 32], stage=3, block='b')
    X = identity_block(X, 3, [32, 32], stage=3, block='c')
    X = identity_block(X, 3, [32, 32], stage=3, block='d')
    X = identity_block(X, 3, [32, 32], stage=3, block='e')
    X = identity_block(X, 3, [32, 32], stage=3, block='f')
    X = identity_block(X, 3, [32, 32], stage=3, block='g')
    X = identity_block(X, 3, [32, 32], stage=3, block='h')
    X = identity_block(X, 3, [32, 32], stage=3, block='i')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [64, 64], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [64, 64], stage=4, block='b')
    X = identity_block(X, 3, [64, 64], stage=4, block='c')
    X = identity_block(X, 3, [64, 64], stage=4, block='d')
    X = identity_block(X, 3, [64, 64], stage=4, block='e')
    X = identity_block(X, 3, [64, 64], stage=4, block='f')
    X = identity_block(X, 3, [64, 64], stage=4, block='g')
    X = identity_block(X, 3, [64, 64], stage=4, block='h')
    X = identity_block(X, 3, [64, 64], stage=4, block='i')

    # AVGPOOL. Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(8, 8))(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X) 
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet56')

    return model

In [5]:
model = ResNet56(input_shape = (32, 32, 3), classes = 10)